In [ ]:
#Install SImple Transformers 

!pip install simpletransformers

In [ ]:
#Importing libraries and loading data
import logging
from simpletransformers.t5 import T5Model, T5Args
import pandas as pd
from simpletransformers.seq2seq import Seq2SeqModel, Seq2SeqArgs
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/news_summary.csv", encoding='latin-1')
df = df[['text','ctext']]
df = df[['ctext','text']].head(1000)
df = df.rename(columns = {'ctext':'input_text' , 'text' : 'target_text'})
df

In [ ]:
#Data Preprocessing for input text
for i in range(1000):
  df.input_text[i] = str(df.input_text[i])
  df.input_text[i] = df.input_text[i].replace("?","") 
  df.input_text[i] = df.input_text[i].replace("/","") 
  df.input_text[i] = df.input_text[i].replace(")","")
  df.input_text[i] = df.input_text[i].replace("(","")
  df.input_text[i] = df.input_text[i].replace("|","")
  df.input_text[i] = df.input_text[i].replace(",","")
  df.input_text[i] = df.input_text[i].replace("'","")
  df.input_text[i] = df.input_text[i].replace("\t","")
  df.input_text[i] = df.input_text[i].replace("\r","") 
  df.input_text[i] = df.input_text[i].replace("\n","")
  df.input_text[i] = df.input_text[i].replace('"',"")
  df.input_text[i] = df.input_text[i].replace('@',"")
  df.input_text[i] = df.input_text[i].replace('#',"")
  df.input_text[i] = df.input_text[i].replace('&',"")
  df.input_text[i] = df.input_text[i].replace('*',"")
  df.input_text[i] = df.input_text[i].replace('!',"")
  df.input_text[i] = df.input_text[i].replace('-',"")
  df.input_text[i] = df.input_text[i].lower()

In [ ]:
#Data Preprocessing for output text
for i in range(1000):
  df.target_text[i] = df.target_text[i].replace("?","") 
  df.target_text[i] = df.target_text[i].replace("/","") 
  df.target_text[i] = df.target_text[i].replace(")","")
  df.target_text[i] = df.target_text[i].replace("(","")
  df.target_text[i] = df.target_text[i].replace("|","")
  df.target_text[i] = df.target_text[i].replace(",","")
  df.target_text[i] = df.target_text[i].replace("'","")
  df.target_text[i] = df.target_text[i].replace("\t","")
  df.target_text[i] = df.target_text[i].replace("\r","") 
  df.target_text[i] = df.target_text[i].replace("\n","")
  df.target_text[i] = df.target_text[i].replace('"',"")
  df.target_text[i] = df.target_text[i].replace('@',"")
  df.target_text[i] = df.target_text[i].replace('#',"")
  df.target_text[i] = df.target_text[i].replace('&',"")
  df.target_text[i] = df.target_text[i].replace('*',"")
  df.target_text[i] = df.target_text[i].replace('!',"")
  df.target_text[i] = df.target_text[i].replace('-',"")
  df.target_text[i] = df.target_text[i].lower()

In [ ]:
#T5 Model requires the task as the prefix
df['prefix'] = 'summarization' 
df = df[['prefix' , 'input_text' , 'target_text']]
df

In [ ]:
#Dropping the Null values from the columns and splitting the data into train and test 
df = df.dropna()
df
from sklearn.model_selection import train_test_split
train , test = train_test_split(df , test_size = 0.2)

In [ ]:
train.shape , test.shape

In [ ]:
#Setting up the logger

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# Configure the model arguments
model_args = T5Args()
model_args.num_train_epochs = 50
model_args.evaluate_generated_text = True
model_args.evaluate_during_training = True
model_args.evaluate_during_training_verbose = True
model_args.overwrite_output_dir = True
model_args.train_batch_size=12
model_args.eval_batch_size = 8
model_args.max_seq_length = 350
model_args.learning_rate = 2e-5
model_args.preprocess_inputs = True
model_args.early_stopping = True
model_args.max_length = 350
model_args.repetition_penalty = 2.5

#Defining the model
model = T5Model(
    "t5",
    "t5-small",
    args=model_args,
    use_cuda = True
)

# Train the model
model.train_model(train, eval_data=test, output_dir = "/content/drive/MyDrive")


In [ ]:
#Evaluating the model on test data
result = model.eval_model(test)

In [ ]:
#Reloading the model to test on custom data 

model_reloaded = T5Model(
    "t5",
    "outputs/best_model",
    args=model_args
)

In [ ]:
print(
    model_reloaded.predict(
        [
          "We humans always need an escape from the reality once in a while. No better therapy than Maldives is truly a paradise on earth! And if you’re wishing to travel somewhere from india during this COVID situations and restrictions then this is the best option in present. It is open for all the Indians and all the safety measures are well taken care of. There are some points to be noted before plan a trip for Maldives. it’s always better to choose the all inclusive package. Choose a resort which is giving you good inclusions and facilities such as food, water activities, spa, privacy for a honeymoon couple, complementary RTPCR test.You need to have a RTPCR test done before 72 hours of your flight date and get a mandatory negative report! Have a travel insurance and make sure you’ve filled all the documents and have a xerox copies of it too. Reach the airport prior 2 hours of your flight timings for the safer side because check in and immigration might take some time. make sure you choose the earliest flight available of the day as your first day won’t be wasted because after arriving in Maldives airport you’ll have to wait for your boat or sea plane for your resort and almost one hour goes here. Sea planes are available only till 4 pm so if you have to catch a sea plane you need to land on Maldives airport  by 2 pm max! "
        ]
                )
)

In [ ]:
#Generating the result using pipeline

from transformers import pipeline
from transformers import AutoTokenizer, T5ForConditionalGeneration, T5Tokenizer
import torch 
import torch.nn.functional as F

model_name = T5ForConditionalGeneration.from_pretrained("outputs/best_model")
tokenizer = AutoTokenizer.from_pretrained("outputs/best_model")

generator = pipeline('summarization', model=model_name, tokenizer= tokenizer)
print("generator done")
text = "We humans always need an escape from the reality once in a while. No better therapy than Maldives is truly a paradise on earth! And if you’re wishing to travel somewhere from india during this COVID situations and restrictions then this is the best option in present. It is open for all the Indians and all the safety measures are well taken care of. There are some points to be noted before plan a trip for Maldives. it’s always better to choose the all inclusive package. Choose a resort which is giving you good inclusions and facilities such as food, water activities, spa, privacy for a honeymoon couple, complementary RTPCR test.You need to have a RTPCR test done before 72 hours of your flight date and get a mandatory negative report! Have a travel insurance and make sure you’ve filled all the documents and have a xerox copies of it too. Reach the airport prior 2 hours of your flight timings for the safer side because check in and immigration might take some time. make sure you choose the earliest flight available of the day as your first day won’t be wasted because after arriving in Maldives airport you’ll have to wait for your boat or sea plane for your resort and almost one hour goes here. Sea planes are available only till 4 pm so if you have to catch a sea plane you need to land on Maldives airport  by 2 pm max! "
results = generator(text, max_length =300)
result = results[0]['summary_text']
print(result)